## Model - DistilBERT

For sentiment analysis, DistilBERT is a good option as it is one of the fastest due to how efficient it is, while retaining most of the accuracy. It has good understanding of context, sarcasm and negation which makes it suitable for such usages for analysing emotions in the text. And given that it’s already pre-trained, it can easily be fine-tuned for excellent performance

In [ ]:
pip install pandas transformers scikit-learn tqdm nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_scheduler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Loading the train dataset
df_train = pd.read_csv("processed_train.csv")
df_test = pd.read_csv("processed_test.csv")

In [ ]:
df_train.isnull().sum()

sentiment           0
processed_review    0
dtype: int64

In [ ]:
blank_counts = (df_train.isna() | df_train.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

sentiment           0
processed_review    0
dtype: int64


/tmp/ipykernel_250/313185806.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (df_train.isna() | df_train.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


In [ ]:
df_test.isnull().sum()

sentiment           0
processed_review    0
dtype: int64

In [ ]:
blank_counts = (df_test.isna() | df_test.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

sentiment           0
processed_review    0
dtype: int64


/tmp/ipykernel_250/2976819833.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (df_test.isna() | df_test.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


In [ ]:
# Ensuring correct data types
df_train['sentiment'] = df_train['sentiment'].astype(int)
df_test['sentiment'] = df_test['sentiment'].astype(int)

In [ ]:
# Defining the Dataset Class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Ensuring that text is a string
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# Initializing the DistilBERT Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
# Preparing the Dataset
dataset_train = TextDataset(df_train['processed_review'].tolist(), df_train['sentiment'].tolist(), tokenizer)
dataset_test = TextDataset(df_test['processed_review'].tolist(), df_test['sentiment'].tolist(), tokenizer)


In [ ]:
# DataLoader
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False)

In [ ]:
# Loading the DistilBERT Model for Sentiment Classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
#  Defining the Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training Function
def train_model(model, train_loader, optimizer, criterion, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

            progress_bar.set_postfix(loss=total_loss / len(train_loader))

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "trained_distilBERT_model.pt")
    print("Model saved successfully!")

# Training the model
train_model(model, train_loader, optimizer, criterion, scheduler)

Epoch 1: 100%|██████████| 32497/32497 [24:55<00:00, 21.72it/s, loss=0.172]


Epoch 1, Loss: 0.1724


Epoch 2: 100%|██████████| 32497/32497 [24:53<00:00, 21.76it/s, loss=0.106]


Epoch 2, Loss: 0.1062


Epoch 3: 100%|██████████| 32497/32497 [24:52<00:00, 21.78it/s, loss=0.0597]


Epoch 3, Loss: 0.0597
Model saved successfully!


In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Evaluating")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

# Evaluating the model
evaluate_model(model, test_loader)

Evaluating: 100%|██████████| 2500/2500 [01:19<00:00, 31.53it/s]


Accuracy: 0.9478
Precision: 0.9478
Recall: 0.9478
F1 Score: 0.9478
